In [10]:
import math  
  
def calculate_key_strength(key_type, T, D, l, b=None, k=None,
                           t=None, C=None, mu=0.15):  
    """  
    Calculate the strength of different types of keys based on input parameters.  
  
    Parameters:  
    - key_type: str, type of key ('flat', 'semicircular', 'wedge', 'tangential', 'end').  
    - T: float, torque applied, Nm.  
    - D: float, diameter of the shaft, mm.  
    - l: float, working length of the key, mm.  
    - b: float, width of the key, mm (required for wedge and tangential keys).  
    - k: float, contact height of key and hub, mm (optional, but required for accurate calculation for flat and semicircular keys).  
    - t: float, working surface width of tangential key, mm (required for tangential key).  
    - C: float, chamfer width of tangential key, mm (required for tangential key).  
    - mu: float, friction coefficient, default 0.15 for steel and cast iron.  
  
    Returns:  
    - string: formatted result with formula and calculated stress.  
    - float: calculated stress value in MPa.  
    """  
    sigma_p = None  
      
    if key_type == 'flat':  
        if k is None:  
            k = 0.4 * D  # Default value for flat key, should be adjusted based on actual design  
        sigma_p = 2 * T / (D * k * l)  
    elif key_type == 'semicircular':  
        # For semicircular keys, k should be taken from a table or design specification  
        if k is None:  
            raise ValueError("k value is required for semicircular key calculations.")  
        sigma_p = 2 * T / (D * k * l)  
    elif key_type == 'wedge':  
        if b is None:  
            raise ValueError("b (key width) is required for wedge key calculations.")  
        sigma_p = 12 * T * (6 * mu * D + b) / (b * l)  
    elif key_type == 'tangential':  
        if b is None or t is None or C is None:  
            raise ValueError("b, t, and C are required for tangential key calculations.")  
        sigma_p = T / ((0.5 * mu + 0.45) * D * l * (t - C))  
    elif key_type == 'end':  
        # Assuming l1 is part of the total key length, but this needs to be specified or calculated  
        # For simplicity, we'll use l as a placeholder for now  
        l1 = l  # Placeholder, should be replaced with actual value  
        sigma_p = 4 * T * D * (l1 - l / D) ** 2 / (D * l)  
    else:  
        return "Invalid key type.", None  
  
    return f"{key_type} Key:\nσp = {sigma_p:.2f} MPa\nFormula: See comments for specific key type", sigma_p  
  
# Example usage  
key_type = 'wedge'  
T = 1410000  # Nm  
D = 50   # mm  
l = 40    # mm  
b = 16    # mm  
result_str, result_val = calculate_key_strength(key_type, T, D, l, b)  
print(result_str)  
print("Calculated Stress:", result_val, "MPa")


wedge Key:
σp = 1612687.50 MPa
Formula: See comments for specific key type
Calculated Stress: 1612687.4999999998 MPa


In [13]:
def calculate_key_stress(key_type, T, D, l, b=None, k=None, t=None, 
                         C=None, mu=0.15, sigma_pp=125):  
    """  
    计算键连接的挤压应力。  
  
    参数:  
    key_type (str): 键的类型 ('平键', '半圆键', '楔键', '切向键', '端面键')。  
    T (float): 转矩，单位 N·mm。  
    D (float): 轴的直径，单位 mm。  
    l (float): 键的工作长度，单位 mm。  
    b (float, 可选): 键的宽度，单位 mm（仅对楔键和切向键有用）。  
    k (float, 可选): 键与轮毂的接触高度，单位 mm（平键和半圆键）。  
    t (float, 可选): 切向键工作面宽度，单位 mm（仅对切向键有用）。  
    C (float, 可选): 切向键倒角宽度，单位 mm（仅对切向键有用）。  
    mu (float, 可选): 摩擦因数，默认为 0.15（可根据材料修改）。  
    sigma_pp (float, 可选): 材料的许用挤压应力，单位 MPa，默认为 125 MPa。  
  
    返回:  
    tuple: 包含计算公式的字符串和计算出的挤压应力 σ_p。  
    """  
    # 根据键的类型选择不同的计算公式  
    if key_type == '平键':  
        # 默认k为0.4h（这里h未给出，实际中需要根据毂的厚度等确定，这里简化为一个假设值）  
        # 这里为了简化，我们使用D的某个比例作为接触高度k的近似  
        k = 0.4 * D / 10  # 假设k与D成比例  
        sigma_p = 2 * T / (D * k * l)  
        formula = f"σ_p = 2T / (D * k * l)"  
    elif key_type == '半圆键':  
        # 半圆键的k值通常从表或标准中取得，这里简化为D的某个比例  
        k = 0.5 * D / 10  # 假设值  
        sigma_p = 2 * T / (D * k * l)  
        formula = f"σ_p = 2T / (D * k * l)"  
    elif key_type == '楔键':  
        sigma_p = 12 * T / (b * l* ((6 * mu * D + b)))  
        formula = f"σ_p = 12T / b * l / ((6μ * D + b) "  
    elif key_type == '切向键':  
        sigma_p = T / ((0.5 * mu + 0.45) * D * l * (t - C))  
        formula = f"σ_p = T / ((0.5μ + 0.45) * D * l * (t - C))"  
    elif key_type == '端面键':  
        sigma_p = 4 * T * D * l / (D**2 * (l - l/D)**2)  
        formula = f"σ_p = 4T * D * l / (D^2 * (l - l/D)^2)"  
    else:  
        raise ValueError("不支持的键类型")  
  
    # 验证强度  
    if sigma_p > sigma_pp:  
        print(f"警告：计算出的挤压应力 {sigma_p:.2f} MPa 大于许用应力 {sigma_pp:.2f} MPa")  
  
    return formula, sigma_p  
  
# 示例使用  
formula, stress = calculate_key_stress('楔键', T=1410000000, D=400, l=380,b=75)  
print(f"公式: {formula}, 挤压应力: {stress/4:.2f} MPa")

警告：计算出的挤压应力 1364.79 MPa 大于许用应力 125.00 MPa
公式: σ_p = 12T / b * l / ((6μ * D + b) , 挤压应力: 341.20 MPa


In [20]:
a=(400000*(1.02**30)-400000)/12/30
print(a)

901.5128712259499
